In [1]:
import numpy as np
import tensorflow as tf
from functools import reduce
import matplotlib.pyplot as plt
sess = tf.InteractiveSession()

In [2]:
num_features = 2
num_samples = 5
state_size = 4
num_of_operations = 3
batch_size = num_samples
max_num_features = 10
datatype = tf.float64


#model placeholders
batchX_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchX")
batchY_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchY")

def add(vec):
    return reduce((lambda x, y: x + y),vec)

def samples_generator(fn, shape, rng, seed=None):
    '''
    Generate random samples for the model:
    @fn - function to be applied on the input features to get the ouput
    @shape - shape of the features matrix (num_samples, num_features)
    @rng - range of the input features to be generated within (a,b)
    @seed  - generation seed
    Outputs a tuple of input and output features matrix
    '''
    x = (rng[1] - rng[0]) * np.random.random_sample(shape) + rng[0]
    y = np.apply_along_axis(fn, 1, x).reshape((-1,1))

    #x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    #y = y.reshape((batch_size, -1))

    return (x, y)


x,y = samples_generator(add, (num_samples, num_features) , (0, 100))

def tf_multiply(inpt):
    return tf.reshape( tf.reduce_prod(inpt, axis = 1), [batch_size, -1])

def tf_add(inpt):
    return  tf.reshape( tf.reduce_sum(inpt, axis = 1), [batch_size, -1])

def tf_stall(a):
    return a


x = tf.Variable(x, dtype=datatype)
y = tf.Variable(y, dtype=datatype)
x.initializer.run()
y.initializer.run()



In [3]:
add    = tf_add(x)
mult   = tf_multiply(x)
stall  = tf_stall(x)
values = tf.concat([add, mult, stall], 1)
print(values.eval())

[[  141.48567826  4589.71820945    91.1102415     50.37543677]
 [  104.33198083  1319.18489855    89.61069175    14.72128909]
 [   75.0898721   1363.1166511     44.36443545    30.72543665]
 [  160.70475372  6408.00707001    87.3163841     73.38836962]
 [  118.4720909   3305.2721499     44.96590423    73.50618667]]


In [4]:
init_state = tf.Variable(np.zeros([batch_size, state_size]), dtype=datatype)
argmax_dum = tf.Variable([2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  dtype=datatype)
W = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=datatype)
b = tf.Variable(np.zeros((state_size)), dtype=datatype)

W2 = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=datatype)
b2 = tf.Variable(np.zeros((num_of_operations)), dtype=datatype)

dummy_matrix = tf.zeros([batch_size, num_features], datatype, name="dummy_constant")

init_state.initializer.run()
argmax_dum.initializer.run()
W.initializer.run()
b.initializer.run()
W2.initializer.run()
b2.initializer.run()
#dummy_matrix.initializer.run()

current_state = init_state
current_input = x
      
input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # Increasing number of columns
next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
current_state = next_state
    
    #calculate softmax and produce the mask of operations
logits  = tf.matmul(next_state, W2) + b2 #Broadcasted addition
softmax = tf.nn.softmax(logits)
softmax_10000 = tf.nn.softmax(logits)
softplus = tf.nn.softplus(logits)
softsign = tf.nn.softsign(logits)
argmax  = tf.argmax(softmax, 1)

#lengths_transposed = tf.expand_dims(argmax, 0)
#onehot  = tf.one_hot(argmax, num_of_operations)
stall_width = tf.shape(stall)[1] - 1 
stall_select = tf.slice(softmax, [0,2], [batch_size,1])
multiples = [1, stall_width]
stall_mask = tf.tile(stall_select, multiples)
mask = tf.concat([softmax, stall_mask], 1)
#mask = tf.cast(mask, tf.bool)
#apply mask
#output = tf.boolean_mask(values,mask)
output = values * mask
nn_soft = tf.nn.softmax(logits)
output_width = tf.shape(output)[1]
#zeros_slice = tf.slice(dummy_matrix, [0,0], [batch_size, max_num_features - output_width])
#dum_plus_input = tf.concat([output, zeros_slice], 1)
#print(dum_plus_input.eval())

'''
print(x.eval())
print("stall_select")
print(stall_select.eval())
print("mask")
print(mask.eval())
print("logits")
print(logits.eval())
print("softmax")
print(softmax.eval())
print("softplus")
print(softplus.eval())
print("softsign")
print(softsign.eval())
print("softmax*10000")
print(softmax_10000.eval())
print("onehot")
print(onehot.eval())
'''
print(mask.eval())
#print(mask.eval())
#print(output.eval())

[[ 0.16628098  0.43735987  0.39635915  0.39635915]
 [ 0.16628689  0.43735153  0.39636158  0.39636158]
 [ 0.16628099  0.43735986  0.39635915  0.39635915]
 [ 0.16628098  0.43735987  0.39635915  0.39635915]
 [ 0.16628098  0.43735987  0.39635915  0.39635915]]


In [5]:
add_softmax   = tf.slice(softmax, [0,0], [batch_size,1])
mult_softmax  = tf.slice(softmax, [0,1], [batch_size,1])
stall_softmax = tf.slice(softmax, [0,2], [batch_size,1])

add_width   = tf.shape(add)[1]
mult_width  = tf.shape(mult)[1]
stall_width = tf.shape(stall)[1]


add_final   = add * add_softmax
mult_final  = mult * mult_softmax
stall_final = stall * stall_softmax

##conact add and mult results with zeros matrix
add_final = tf.concat([add_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - add_width])], 1) 
mult_final = tf.concat([mult_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - mult_width])], 1) 

output_final = add_final + mult_final + stall_final 

print(add_width.eval())
print(mult_width.eval())
print(stall_width.eval())


print(add.eval())
print(mult.eval())
print(stall.eval())

print(add_softmax.eval())
print(mult_softmax.eval())
print(stall_softmax.eval())

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

print(output_final.eval())

1
1
2
[[ 141.48567826]
 [ 104.33198083]
 [  75.0898721 ]
 [ 160.70475372]
 [ 118.4720909 ]]
[[ 4589.71820945]
 [ 1319.18489855]
 [ 1363.1166511 ]
 [ 6408.00707001]
 [ 3305.2721499 ]]
[[ 91.1102415   50.37543677]
 [ 89.61069175  14.72128909]
 [ 44.36443545  30.72543665]
 [ 87.3163841   73.38836962]
 [ 44.96590423  73.50618667]]
[[ 0.16628098]
 [ 0.16628689]
 [ 0.16628099]
 [ 0.16628098]
 [ 0.16628098]]
[[ 0.43735987]
 [ 0.43735153]
 [ 0.43735986]
 [ 0.43735987]
 [ 0.43735987]]
[[ 0.39635915]
 [ 0.39636158]
 [ 0.39635915]
 [ 0.39635915]
 [ 0.39635915]]
[[ 23.52637723   0.        ]
 [ 17.34904059   0.        ]
 [ 12.48601801   0.        ]
 [ 26.72214393   0.        ]
 [ 19.69965537   0.        ]]
[[ 2007.3585534      0.        ]
 [  576.94753411     0.        ]
 [  596.17250686     0.        ]
 [ 2802.60513069     0.        ]
 [ 1445.59339346     0.        ]]
[[ 36.112378    19.96676537]
 [ 35.51823536   5.8349534 ]
 [ 17.58425011  12.17830808]
 [ 34.6086479   29.0881519 ]
 [ 17.82264764 

In [6]:

def custom_softmax(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = tf.log(sum_exponents)
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( inpt / soft_max, [batch_size, -1])

def custom_softmax2(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = sum_exponents
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( tf.exp(inpt) / soft_max, [batch_size, -1])
x_cust = custom_softmax(x)
x_cust2 = custom_softmax2(x)
print(x.eval())
#print(x_cust2.eval())
print(tf.nn.softmax(x*10000).eval())

[[ 91.1102415 ]
 [ 89.61069175]
 [ 44.36443664]
 [ 87.31638499]
 [ 73.50618667]]
[[  3.70403539e+39]
 [  8.26854216e+38]
 [  1.85024835e+19]
 [  8.33727607e+37]
 [  8.38168345e+31]]
[[ 91.1102415   50.37543677]
 [ 89.61069175  14.72128909]
 [ 44.36443545  30.72543665]
 [ 87.3163841   73.38836962]
 [ 44.96590423  73.50618667]]
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]


In [7]:
math_error = tf.multiply(tf.constant(0.5, dtype=datatype), tf.square(tf.subtract(tf.reduce_sum(output, axis = 1, name="red_output") , batchY_placeholder, name="sub_otput_batchY"), name="squar_error"), name="mult_with_0.5")

total_loss = tf.reduce_sum(math_error, name="red_total_loss")

grads = tf.gradients(total_loss, [W,b,W2,b2], name="comp_gradients")
train_step = tf.train.AdamOptimizer(0.1, name="AdamOpt").minimize(total_loss, name="min_loss")

In [20]:
def calc_loss(output):
    reduced_output = tf.reshape( tf.reduce_sum(output, axis = 1, name="red_output"), [batch_size, -1])
    math_error = tf.multiply(tf.constant(0.5, dtype=datatype), tf.square(tf.subtract(reduced_output , y, name="sub_otput_batchY"), name="squar_error"), name="mult_with_0.5")
    
                             
    total_loss = tf.reduce_sum(math_error, name="red_total_loss")
    return total_loss, math_error


loss, mathh = calc_loss(output_final)

In [21]:
mathh.eval()

array([[ 1945.47839573],
       [  531.31778264],
       [  563.33121096],
       [ 2732.3193207 ],
       [ 1393.77845534]])

In [10]:
#sess.close()